In [1]:
# Standard Library Imports
import os
import time
import pyautogui

# Web Automation Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

# Data Handling Imports
import pandas as pd

import keyring
import sys
import yfinance as yf
from datetime import datetime

# HTTP Requests Imports
import requests

In [2]:
def download_stock_data(ticker, start_date):
    # Define the end date as today
    end_date = datetime.today().strftime('%Y-%m-%d')

    # Download stock data using yfinance
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    
    # Get the current date for file naming (most recent working day)
    today = datetime.today().strftime('%Y%m%d')
    
    # Define the filename as {ticker}-YYYYMMDD.csv
    filename = f"{ticker}-{end_date}.csv"
    
    # Save to a CSV file (SAS-compatible)
    stock_data.to_csv(filename)
    
    print(f"Data for {ticker} from {start_date} to {end_date} has been saved to {filename}")

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python script.py <TICKER> <START_DATE>")
        print("Example: python script.py AAPL 2023-01-01")
    else:
        ticker = sys.argv[1]
        start_date = sys.argv[2]
        download_stock_data(ticker, start_date)

[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['-F']: Exception('%ticker%: No timezone found, symbol may be delisted')



Data for -f from /Users/szymonczuszek/Library/Jupyter/runtime/kernel-f74ce020-bab4-43a5-b496-224df946a2a5.json to 2024-10-10 has been saved to -f-2024-10-10.csv


In [3]:
# the service is just a namespace for your app
service_id = "SAS"

user_email = "szymon.czuszek@edu.uekat.pl"

# keyring.set_password(service_id, user_email, 'your password')

password = keyring.get_password(service_id, user_email) # retrieve password

In [4]:
sas_code = '''
DATA AAPL;
INFILE "/home/u63805106/Stock Data/AAPL-20241008.csv" DSD MISSOVER 
FIRSTOBS=2;
INPUT Date : yymmdd10. Open High Low Close AdjClose Volume;
FORMAT Date yymmdd10.; /* Format the date for display */
RUN;
'''

filename = "AAPL-2024-10-08.csv"

In [5]:
# Record the start time
start_time = time.time()

# Initialize WebDriver (assuming Chrome)
driver = webdriver.Chrome()

# Open rpachallenge.com
driver.get("https://welcome.oda.sas.com/?event=logout&eventSource=eu-west-1a")
driver.maximize_window()

# Click the Sign In button
sign_in_button = driver.find_element(By.CSS_SELECTOR, ".SIbutton")
sign_in_button.click()

# Enter your email using XPath
email_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@id='emailOrAccountName']"))
)
email_input.send_keys(user_email)  # Replace with your email

# Enter your password using XPath
password_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@id='password']"))
)
password_input.send_keys(password)  # Replace with your password

# Select the checkbox using XPath
checkbox = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@id='agreeToLicenseAndTerms']"))
)
if not checkbox.is_selected():
    checkbox.click()
    
# Click the "Sign In" button using XPath
sign_in_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[@mat-based-button='']"))
)
sign_in_button.click()

# Optionally, wait to observe the result
time.sleep(2)

# Store the current window handle
current_window = driver.current_window_handle

# Wait for the "Deploy" button to appear
deploy_button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//span[contains(@class, 'deploy')]"))
)

# Click the "Deploy" button
deploy_button.click()

# Wait for the new tab to open and switch to it
WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

# Switch to the new tab
for window_handle in driver.window_handles:
    if window_handle != current_window:
        driver.switch_to.window(window_handle)
        break
        
# Wait for the page to fully load
time.sleep(10)

# Wait for the element to be visible
files_home_element = WebDriverWait(driver, 60).until(
    EC.visibility_of_element_located((By.XPATH, "//span[contains(text(), 'Files (Home)')]"))
)

pyautogui.typewrite(sas_code)

# Create an ActionChains object
actions = ActionChains(driver)

# Perform a double-click on the element
actions.double_click(files_home_element).perform()

# Wait a moment to ensure the focus change
time.sleep(5)

files_stock_element = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.XPATH, "//span[contains(text(), 'Stock Data')]"))
    )

# Create an ActionChains object
actions = ActionChains(driver)

# Perform a double-click on the "Stock Data" button
actions.double_click(files_stock_element).perform()

# Wait for the element with class 'sasUploadIcon' to be visible
files_upload_element = WebDriverWait(driver, 60).until(
    EC.visibility_of_element_located((By.XPATH, "//span[contains(@class, 'sasUploadIcon')]"))
)

# Click the element once it's visible
files_upload_element.click()

from selenium.webdriver.common.keys import Keys

# Press Tab to navigate to the next focusable element (this may depend on the page structure)
actions.send_keys(Keys.TAB).perform()

# Wait a moment to ensure the focus change
time.sleep(5)

# Press Enter to activate the focused element (which should be the "Deploy" button)
actions.send_keys(Keys.ENTER).perform()

# Wait a moment to ensure the focus change
time.sleep(5)

# Press the Tab key
pyautogui.press('tab')

# Wait a moment to ensure the focus change
time.sleep(5)

pyautogui.typewrite(filename)

# Wait a moment to ensure the focus change
time.sleep(5)

# Press the Tab key
pyautogui.press('tab')

# Wait a moment to ensure the focus change
time.sleep(5)

# Press the Tab key
pyautogui.press('tab')

# Wait a moment to ensure the focus change
time.sleep(5)

# Press the Arrow Down key
pyautogui.press('down')

# Wait a moment to ensure the focus change
time.sleep(5)

# After navigating, press Enter to select the highlighted option
pyautogui.press('enter')

# Wait a moment to ensure the focus change
time.sleep(5)

# Press Enter to activate the focused element (which should be the "Deploy" button)
actions.send_keys(Keys.ENTER).perform()

# Press Tab to navigate to the next focusable element (this may depend on the page structure)
actions.send_keys(Keys.TAB).perform()

# Press Tab to navigate to the next focusable element (this may depend on the page structure)
actions.send_keys(Keys.TAB).perform()

# Press Tab to navigate to the next focusable element (this may depend on the page structure)
actions.send_keys(Keys.TAB).perform()

# Press Tab to navigate to the next focusable element (this may depend on the page structure)
actions.send_keys(Keys.TAB).perform()

# Wait a moment to ensure the focus change
time.sleep(5)

# <span class="dijitReset dijitStretch dijitButtonContents" data-dojo-attach-point="titleNode,focusNode" role="button" aria-labelledby="perspectiveTabContainer_tabsBC_tab0_formatCodeBtn_label" tabindex="-1" title="Format code" id="perspectiveTabContainer_tabsBC_tab0_formatCodeBtn" style="user-select: none;"><span class="dijitReset dijitInline dijitIcon sasFormatIcon" data-dojo-attach-point="iconNode"></span><span class="dijitReset dijitToggleButtonIconChar">●</span><span class="dijitReset dijitInline dijitButtonText dijitDisplayNone" id="perspectiveTabContainer_tabsBC_tab0_formatCodeBtn_label" data-dojo-attach-point="containerNode">Format Code</span></span>

# Wait for the "Format Code" button to be visible
format_button = WebDriverWait(driver, 60).until(
    EC.visibility_of_element_located((By.XPATH, "//span[@aria-labelledby='perspectiveTabContainer_tabsBC_tab0_formatCodeBtn_label']"))
)

# Click the "Format Code" button
format_button.click()

# Wait a moment to ensure the focus change
time.sleep(5)

# <span class="dijitReset dijitStretch dijitButtonContents" data-dojo-attach-point="titleNode,focusNode" role="button" aria-labelledby="perspectiveTabContainer_tabsBC_tab0_submitBtn_label" tabindex="-1" title="Run all or selected code (F3)" aria-disabled="false" id="perspectiveTabContainer_tabsBC_tab0_submitBtn" style="user-select: none;"><span class="dijitReset dijitInline dijitIcon sasSubmitIcon" data-dojo-attach-point="iconNode"></span><span class="dijitReset dijitToggleButtonIconChar">●</span><span class="dijitReset dijitInline dijitButtonText dijitDisplayNone" id="perspectiveTabContainer_tabsBC_tab0_submitBtn_label" data-dojo-attach-point="containerNode">Run all or selected code (F3)</span></span>

# Wait for the element to be visible

run_button = WebDriverWait(driver, 60).until(
    EC.visibility_of_element_located((By.XPATH, "//span[@aria-labelledby='perspectiveTabContainer_tabsBC_tab0_submitBtn_label']"))
)

run_button.click()